# WhisperTS JSON vers SRT

In [1]:
import re,codecs,json
import pandas as pd

In [2]:
fTranscript="FrAr-00-22.json"
fTranscript="FrAr.json"
pause=0.200
with open(fTranscript) as inFile:
    whisper=json.load(inFile)

In [3]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

In [4]:
def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

In [5]:
def getWhisper(elements):
    lElements=[]
    if elements=="segments":
        for element in whisper[elements]:
            lElements.append((second2timestamp(element["start"]),
                              second2timestamp(element["end"]),
                              element["text"],
                              element["confidence"]))
    elif elements=="words":
        for segment in whisper["segments"]:
            for element in segment[elements]:
                lElements.append((second2timestamp(element["start"]),
                                  second2timestamp(element["end"]),
                                  element["text"],
                                  element["confidence"]))        
    return lElements

In [6]:
tMots=getWhisper("words")
dfMots=pd.DataFrame(tMots,columns="start end mot confidence".split(" "))
dfMots

,start,end,mot,confidence
0,"00:00:04,370","00:00:05,410","Spreadbury,",0.381
1,"00:00:05,490","00:00:06,049",il,0.927
2,"00:00:06,049","00:00:06,290",sera,0.994
3,"00:00:06,290","00:00:06,929",encadré,0.996
4,"00:00:06,929","00:00:07,030",par,0.992
...,...,...,...,...
24280,"01:47:15,029","01:47:15,989",Grosse.,0.001
24281,"01:47:17,130","01:47:17,210",Grosse.,0.000
24282,"01:47:17,210","01:47:17,430",Grosse.,0.007
24283,"01:47:18,029","01:47:18,310",Grosse.,0.004


In [7]:
dfSegments=pd.DataFrame(getWhisper("segments"),columns="start end mot confidence".split(" "))
dfSegments

,start,end,mot,confidence
0,"00:00:04,370","00:00:11,650","Spreadbury, il sera encadré par un Australien...",0.843
1,"00:00:13,279","00:00:22,410","M. Spreadbury, c'est le plus âgé des arbitres...",0.809
2,"00:00:22,429","00:00:29,269",c'est lui qui dirigeait le France-Argentine g...,0.895
3,"00:00:29,410","00:00:39,049",Après avoir demandé si tout le monde était pr...,0.911
4,"00:00:40,390","00:00:43,070","Allez, de suite dans le camp, Argentin...",0.896
...,...,...,...,...
4649,"01:47:15,029","01:47:15,989",Grosse.,0.001
4650,"01:47:17,130","01:47:17,210",Grosse.,0.000
4651,"01:47:17,210","01:47:17,430",Grosse.,0.007
4652,"01:47:18,029","01:47:18,310",Grosse.,0.004


In [8]:
second2timestamp(3665.547)

'01:01:05,547'

In [9]:
srtLines=[]
for i,(start,end,mot,confidence) in enumerate(tMots):
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


In [10]:
with codecs.open(fTranscript.replace(".json","-mots.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")

In [11]:
srtPhrases=[]
srtMots=[]
bPhrase=True
pStart=0
pEnd=0
for i,(start,end,mot,confidence) in enumerate(tMots):
    
    if bPhrase:
        print ("start",start)
        pStart=start
        srtMots=[]
        bPhrase=False
    m=re.search(r"[.?!]$",mot)
    if m:
        print("end",end)
        pEnd=end
        srtMots.append(mot)
        pPhrase=" ".join(srtMots)
        srtPhrases.append((pStart,pEnd,pPhrase))
        bPhrase=True
    elif not bPhrase:
        srtMots.append(mot)

start 00:00:04,370
end 00:00:07,870
start 00:00:07,990
end 00:00:11,009
start 00:00:11,310
end 00:00:11,650
start 00:00:13,279
end 00:00:13,460
start 00:00:13,580
end 00:00:29,269
start 00:00:29,410
end 00:00:39,049
start 00:00:40,390
end 00:00:43,070
start 00:00:43,130
end 00:00:53,090
start 00:00:53,789
end 00:01:00,530
start 00:01:00,729
end 00:01:17,049
start 00:01:19,409
end 00:01:34,230
start 00:01:34,230
end 00:01:40,319
start 00:01:40,799
end 00:01:46,640
start 00:01:46,879
end 00:01:53,219
start 00:01:53,379
end 00:01:59,219
start 00:01:59,700
end 00:02:02,459
start 00:02:02,459
end 00:02:04,640
start 00:02:04,640
end 00:02:11,360
start 00:02:12,300
end 00:02:15,340
start 00:02:16,520
end 00:02:17,219
start 00:02:20,990
end 00:02:21,669
start 00:02:21,830
end 00:02:23,250
start 00:02:24,430
end 00:02:31,000
start 00:02:34,039
end 00:02:35,919
start 00:02:36,080
end 00:02:36,539
start 00:02:41,340
end 00:02:42,259
start 00:02:42,259
end 00:02:52,319
start 00:02:52,560
end 00:03

In [12]:
srtPhrases


[('00:00:04,370',
  '00:00:07,870',
  'Spreadbury, il sera encadré par un Australien, M.'),
 ('00:00:07,990', '00:00:11,009', 'Dickinson, et par un Néo-Zélandais, M.'),
 ('00:00:11,310', '00:00:11,650', 'Lawrence.'),
 ('00:00:13,279', '00:00:13,460', 'M.'),
 ('00:00:13,580',
  '00:00:29,269',
  "[*] Spreadbury, c'est le plus âgé des arbitres de cette Coupe du Monde, le plus expérimenté, [*] et petit signe peut-être du destin, c'est lui qui dirigeait le France-Argentine gagné par l'équipe de France au mois de novembre dernier, ici même au Stade de France."),
 ('00:00:29,410',
  '00:00:39,049',
  "[*] Après avoir demandé si tout le monde était prêt, le coup d'envoi de la Coupe du Monde 2007 est donné par David Skrela !"),
 ('00:00:40,390',
  '00:00:43,070',
  'Allez, de suite dans le camp, [*] Argentin...'),
 ('00:00:43,130',
  '00:00:53,090',
  'Avec un départ de Roncero, [*] Pichot a appelé le ballon [*] pour Hernandez, [*] avec le pied [*] droit, [*] pour trouver une très jolie touche

In [13]:
srtPhrasesLines=[]
for i,(start,end,phrase) in enumerate(srtPhrases):
    n=i+1
    srtPhrasesLines.append(str(n))
    srtPhrasesLines.append(start+" --> "+end)
    srtPhrasesLines.append(phrase)
    srtPhrasesLines.append("")
    
with codecs.open(fTranscript.replace(".json","-phrases.srt"),"w",encoding="utf8") as outFile:
    for srtPhrasesLine in srtPhrasesLines:
        outFile.write(srtPhrasesLine+"\n")

In [14]:
srtTours=[]
srtMots=[]
bTour=True
prevEnd=0
pStart=0
pEnd=0
for i,(start,end,mot,confidence) in enumerate(tMots):
    sStart=timestamp2second(start)
    sEnd=timestamp2second(end)
    if sStart-prevEnd>pause and prevEnd!=0:
        pEnd=prevEnd
        pTour=" ".join(srtMots)
        srtTours.append((pStart,pEnd,pTour))
        pStart=sStart
        prevEnd=sEnd
        srtMots=[mot]
        bPhrase=True
    else:
        srtMots.append(mot)
        prevEnd=sEnd
pTour=" ".join(srtMots)
srtTours.append((pStart,pEnd,pTour))

In [15]:
srtToursLines=[]
for i,(start,end,phrase) in enumerate(srtTours):
    n=i+1
    srtToursLines.append(str(n))
    srtToursLines.append(second2timestamp(start)+" --> "+second2timestamp(end))
    srtToursLines.append(phrase)
    srtToursLines.append("")
    
with codecs.open(fTranscript.replace(".json","-tours.srt"),"w",encoding="utf8") as outFile:
    for srtToursLine in srtToursLines:
        outFile.write(srtToursLine+"\n")